# Metadata extraction and preparation

In [3]:
import os
import json

     |████████████████████████████████| 3.2 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 193 kB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 205 kB/s eta 0:00:01
     |████████████████████████████████| 323 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.3 MB 310 kB/s eta 0:00:01
     |████████████████████████████████| 81 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.9 MB/s  eta 0:00:01
  Using cached pandas-1.3.4-cp39-cp39-macosx_10_9_x86_64.whl (11.6 MB)
     |████████████████████████████████| 17.7 MB 287 kB/s eta 0:00:011
  Using cached rdflib_jsonld-0.6.2-py2.py3-none-any.whl (4.0 kB)
     |████████████████████████████████| 636 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████|

  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
     |████████████████████████████████| 289 kB 13.6 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=d0b37669198a8863cdc7536eaa5404a0ece3dc9224307803587b567938096eb3
  Stored in directory: /Users/aiglesias/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
  Attempting uninstall: Click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.1
    Uninstalling pandas-1.5.1:
    

## Running SOMEF

SOMEF is a tool that automatically extracts relevant information from README files of GitHub/GitLab repositories and saves it as JSON files. We run this tool to extract the metadata from all repositories in a GitHub organisation.

In [9]:
!pip3 install somef
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw-1.4
!python -m nltk.downloader punkt
!python -m nltk.downloader stopwords
!somef configure -a

/Users/aiglesias/miniconda3/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aiglesias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/aiglesias/miniconda3/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aiglesias/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aiglesias/miniconda3/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may r

Once installed and configured, we run the tool to extract the metadata of Mapeathor from [https://github.com/oeg-upm/mapeathor](https://github.com/oeg-upm/mapeathor). To extract every repository in an organisation, the same process needs to be repeated for every repository URL. We show here the extraction of one to exemplify the process.

In [10]:
!somef describe -r https://github.com/oeg-upm/mapeathor -o ../data/somef-data/mapeathor.json

/Users/aiglesias/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
SOftware Metadata Extraction Framework (SOMEF) Command Line Interface
04-Jul-23 17:34:48-INFO-Loading Repository https://github.com/oeg-upm/mapeathor Information....
04-Jul-23 17:34:48-DEBUG-Starting new HTTPS connection (1): api.github.com:443
04-Jul-23 17:34:48-DEBUG-https://api.github.com:443 "GET /repos/oeg-upm/mapeathor HTTP/1.1" 200 1465
04-Jul-23 17:34:48-INFO-Remaining GitHub API requests: 59 ### Next rate limit reset at: 2023-07-04 18:34:48
04-Jul-23 17:34:48-DEBUG-Starting new HTTPS connection (1): api.github.com:443
04-Jul-23 17:34:48-DEBUG-https://api.github.com:443 "GET /repos/oeg-upm/mapeathor/languages HTTP/1.1" 200 53
04-Jul-23 17:34:48-INFO-Remaining GitHub API requests

## Input JSON preparation

Once all desired repositories are processed, we take the list of json files, one corresponding to one repository, remove from the list the repositories that contain ontologies or websites, and merge the remaining ones into one JSON file.

### Clear input JSONs
Removal of names of GitHub repositories containing webpages or ontologies from the list of repositories within the [oeg-upm](https://github.com/oeg-upm/) organisation.

In [2]:
path = '/Users/aiglesias/GitHub/oeg-software-graph/data/somef-data/single-json/'
json_file_names = os.listdir(path)

In [4]:
files_to_delete = []
for file in json_file_names:
    if "web" in file or "ontology" in file:
        files_to_delete.append(file)

clean_json = [file for file in json_file_names if file not in files_to_delete]
clean_json

['oeg-upm_weather1_2023-06-28.json',
 'oeg-upm_bimerr-core_2023-06-28.json',
 'oeg-upm_MIRROR_2023-06-28.json',
 'oeg-upm_bimerr-metadata_2023-06-28.json',
 'oeg-upm_pcake_2023-06-28.json',
 'oeg-upm_cogito-coppola_2023-06-28.json',
 'oeg-upm_mappingpedia-userinterface_2023-06-28.json',
 'oeg-upm_terminology-extractor-incibe_2023-06-28.json',
 'oeg-upm_AttentionRankLib_2023-06-28.json',
 'oeg-upm_ckanext-federgob_2023-06-28.json',
 'oeg-upm_saref-ext_2023-06-28.json',
 'oeg-upm_LabSensingArduino_2023-06-28.json',
 'oeg-upm_esuk_2023-06-28.json',
 'oeg-upm_bimerr-materials_2023-06-28.json',
 'oeg-upm_hola-si-protocol_2023-06-28.json',
 'oeg-upm_termlex_2023-06-28.json',
 'oeg-upm_CODICE-extractor_2023-06-28.json',
 'oeg-upm_ssspotter_2023-06-28.json',
 'oeg-upm_OnToology-view-mock_2023-06-28.json',
 'oeg-upm_Wikidata-class-diagram-generator_2023-06-28.json',
 'oeg-upm_ontologia-ciberseguridad_2023-06-28.json',
 'oeg-upm_tada-map-score_2023-06-28.json',
 'oeg-upm_drugs4covid19-nlp_2023-0

### Merge individual json into one
The selected JSON files are merged into one file to facilitate posterior construction of the knowledge graph.

In [ ]:
merged_json = []
for file in clean_json:
    filename = path + file
    with open(filename, 'r') as infile:
        #print(json.load(infile))
        merged_json.append(json.load(infile))
    
    with open('/Users/aiglesias/GitHub/oeg-software-graph/data/somef.json', 'w') as out_json:
        json.dump(merged_json, out_json)